In [1]:
import pandas as pd
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
import numpy as np
import matplotlib.pyplot as plt
import os

In [2]:
# if NOT working in colab
data_dir = './data'

# if working in colab
# data_dir = './'

In [3]:
df_skill_type = pd.read_csv(os.path.join(data_dir, 'NOC_skilltype.csv'))
df_major_group = pd.read_csv(os.path.join(data_dir, './NOC_majorgroup.csv'))
df_minor_group = pd.read_csv(os.path.join(data_dir, './NOC_minorgroup.csv'))
df = pd.read_csv(os.path.join(data_dir, './noc_data_get_byws_dealing_slash.csv'))

In [4]:
# pad missing digits from noc codes
df['Noc_code'] = df['Noc_code'].apply(lambda x: '{0:0>4}'.format(x))

# Text Preprocessing

## Find unusual characters

In [5]:
ocurrences = 0

def find_character(string, char):
    global ocurrences
    for occ in string.split(';'):
        if char in occ:
            ocurrences += 1
        
df['job_title'].apply(find_character, args=('.'))

print(ocurrences)

34


In [6]:
# key is abbreviation, value is expanded occupation
abbreviations_map = {}
STRIP_ABBREVIATIONS = True

def handle_single_quotes(text):
    """
    handle plurals, which are the main use of the single quote. Afterwards, drop all other single quotes
    """
    text = text.replace("s'", '').replace("'s", '')
    return text.replace("'", '')

def handle_parentheses(text, strip_abbrev):
    """
    Parentheses seem to fall into two general cases in the VAST majority of instances:
    1. Indicates an abbreviation
    2. Indicates an exception, by using keywords such as "except" or "non"
    """
    parentheses_idx = 0
    split = text.split("(")
    for i, substr in enumerate(split):
        if ')' in substr:
            parentheses_idx = i
            break
    
    # fragment before the fragment with the paren.
    str1 = split[parentheses_idx-1].strip()
    assert not ')' in str1
    
    # fragment w parenthesis
    str2 = split[parentheses_idx].split(")")[0].strip()
    
    if 'except' in str2 or 'non' in str2:
        text = text.replace(str2, '')
        # TODO, do something with exceptions
        
    else:
        # take the shorter string as the abbreviation
        ab, ex = (str1, str2) if len(str1) < len(str2) else (str2, str1)
    
        # save abbreviation
        abbreviations_map[ab] = ex
        
        # remove the found abbreviation from job title
        if strip_abbrev:
            text = text.replace(ab, '')

    # remove parentheses, leading and trailing whitespace 
    text = text.replace('(','').replace(')','').strip()
    
    return text

def preprocess_text(text, strip_abbrev=False):
    
    # handle slashes
    text = text.replace("/", ' ')
    
    # remove redundant semi-colons
    text = text.strip(';')
    
    # hyphens are semantic noise, remove
    text = text.replace('-', ' ')
    
    # handle '
    if "'" in text:
        text = handle_single_quotes(text)
    
    # handle ,
    text = text.replace(",", '')
    
    # handle .
    text = text.replace(".", '')
    
    # handle parentheses, only one check necessary since we already verified they are all paired with corresponding ')'
    if "(" in text:
        text = handle_parentheses(text, strip_abbrev=strip_abbrev)
    
    # remove leading and trailing whitespace
    text = text.strip()
    
    # normalize case
    return text.lower()

# Unpack all sample job titles in original df

In [7]:
all_job_samples = {}

def extract_job_samples(row):
    NOC_code = int(row['Noc_code'])
    
    # split jobs contained in row by ';' and .replace('-', '; ') is for '-', .replace('-', '; ')
    # REVISE WHETHER TO KEEP - separation. logic is that lieutenant-governor can be described as lieutenant governer, no hyphen
    # make unique set
    # strip extra characters 
    # and take nonempty elements
    jobs = [
        j for j in  row['job_title'].split(';')
        if (j != '' and j != ' ')
    ]
    
    # change gendered entries such as 'chairman/woman' into separate samples, 'chairman', 'chairwoman'
    for idx, job in enumerate(jobs):
        if 'man/woman' in job:
            # change original entry to 'job(man)', then append job(woman) to end of list
            jobs[idx] = job.replace('man/woman', 'man')
            jobs.append(job.replace('man/woman', 'woman'))
        if 'men/women' in job:
            jobs[idx] = job.replace('men/women', 'men')
            jobs.append(job.replace('men/women', 'women'))
        if 'boy/girl' in job:
            jobs[idx] = job.replace('boy/girl', 'boy')
            jobs.append(job.replace('boy/girl', 'girl'))
        if 'master/mistress' in job:
            jobs[idx] = job.replace('master/mistress', 'master')
            jobs.append(job.replace('master/mistress', 'mistress'))
        if 'host/hostess' in job:
            jobs[idx] = job.replace('host/hostess', 'host')
            jobs.append(job.replace('host/hostess', 'hostess'))
        if 'waiter/waitress' in job:
            jobs[idx] = job.replace('waiter/waitress', 'waiter')
            jobs.append(job.replace('waiter/waitress', 'waitress'))
            
    # remove duplicate entries
    jobs = set(jobs)
    
    # parse counts of each job
    row['n_sample_jobs'] = len(jobs)
    
    # iterate through job and add to dictionary
    for j in jobs:
        
        if j not in all_job_samples:
            all_job_samples[j] = NOC_code

        # safe check, if job appears more than once, clause will print the both NOC Codes
        else:
            if all_job_samples[j] != NOC_code:
                print(j, 'repeated', all_job_samples[j], NOC_code)
    
    return row

def parse_1(row):
    # get info from first digit of 4 digit code
    row['1_digit_target'] = int(str(row['Noc_code'])[0])
    row['1_digit_group'] = df_skill_type[df_skill_type['skilltype_code'] == row['1_digit_target']]['skilltype_title']
        
    return row

def parse_2(row):
    # get info from first 2 digits of 4 digit code
    
    # check if NOC code is long enough for parsing
    if len(str(row['Noc_code'])) > 1:
        row['2_digit_target'] = int(str(row['Noc_code'])[:2])
        row['2_digit_group'] = df_major_group[df_major_group['majorgroup_code'] == '\'' + str(row['2_digit_target'])]['majorgroup_title']
        
    else:
        row['2_digit_target'] = 'NA'
        row['2_digit_group'] = 'NA'
    
    return row

def parse_3(row):
    # get info from first 3 digits of 4 digit code
    
    # check if NOC code is long enough for parsing
    if len(str(row['Noc_code'])) > 2:
        row['3_digit_target'] = int(str(row['Noc_code'])[:3])
        row['3_digit_group'] = df_minor_group[df_minor_group['minorgroup_code'] == '\'' + str(row['3_digit_target'])]['minorgroup_title']
        
    else:
        row['3_digit_target'] = 'NA'
        row['3_digit_group'] = 'NA'
        
    return row

In [10]:
# Do once, if 'noc_code' column already dropped, except to skip action
try:
    df = df.apply(parse_1, axis = 1)
    df = df.apply(parse_2, axis = 1)
    df = df.apply(parse_3, axis = 1)
    df = df.apply(extract_job_samples, axis = 1)
except KeyError:
    pass

# Do same with descriptions

In [11]:
all_descriptions = {}
desc_counts = []
def unpack_descriptions(row):
    # unpack all descriptions from a row and
    duty = row['main_duties']
    desc_counts.append(0)
    
    # split duty field into separate duties and remove initial generic blurb
    for description in duty.strip('-').split(';'):
        if 'duties' not in description:
            all_descriptions[description] = row['Noc_code']
            desc_counts[-1] += 1
            
    return row

df = df.apply(unpack_descriptions, axis = 1)

# Make training dataframe and drop duplicates

In [16]:
train_df = pd.DataFrame(dict(all_job_samples).items(), columns = ['input', 'code'])
print("Samples before dropping duplicates", len(train_df))
train_df['input'] = train_df['input'].apply(preprocess_text, args = (STRIP_ABBREVIATIONS,))
train_df = train_df.drop_duplicates()
print("Samples after dropping duplicates", len(train_df))

Samples before dropping duplicates 29741
Samples after dropping duplicates 29442


# Grab sample to see if preprocessing worked

In [17]:
def check(string):
    try:
        assert '.' not in string \
            and ',' not in string \
                and ')' not in string \
                    and '(' not in string \
                        and '-' not in string \
                            and ';' not in string \
                                and '/' not in string \
                                    and '\'' not in string
    except AssertionError:
        print(string)

In [19]:
train_df['input'].apply(check)
display(train_df.sample(20))

,input,code
9599,contract clerk law,4211
23706,rubber cord splicer,9423
16557,spinning frame fixer textile manufacturing,7311
21470,scale assembly forewoman,9227
13561,food and beverage services attendant,6711
22096,leaching operator primary metal processing,9411
17597,electric household appliance installer residen...,7441
20082,lawn mower,8612
14974,mine electrician,7242
9582,outside quebec,4211


In [20]:
def first_n_digits(string, n=4):
    
    # if default number of digits desired, don't do anything
    if n == 4:
        return string
    
    # else pad left with zeros until 4 digits reached
    padded_str = '{0:0>4}'.format(string)
    return padded_str[:n]

In [25]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

# For window size calculation, get mean length of input

In [26]:
np.mean([len(x.split()) for x in train_df['input']])

3.511717953943346

## Start Doc2vec code

In [27]:
#Import all the dependencies
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize
import nltk
from tqdm import tqdm
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\augus\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [28]:
corpus = list(train_df['input'])
tagged_data = [TaggedDocument(words=word_tokenize(item.lower()), tags=[str(i)]) for i, item in enumerate(corpus)]

In [29]:
TRIAL_NAME = 'trial_9'
curr_model_name = "{}.model".format(TRIAL_NAME)

epochs = 4096 # training cycles
vec_size = 32 # specific to doc2vec, size of the output vector
alpha = 0.001 # learning rate
window = 3
min_count = 2
min_alpha = 0.00025

try:
    assert not os.path.exists(curr_model_name), "Model {} already exists! Update model output name".format(curr_model_name)


    model = Doc2Vec(vector_size=vec_size,
                    alpha=alpha, 
                    window=window,
                    min_alpha=min_alpha,
                    min_count= min_count,
                    dm=1)

    model.build_vocab(tagged_data)

    for epoch in tqdm(range(epochs)):
        model.train(tagged_data,
                    total_examples=model.corpus_count,
                    epochs=model.epochs)
        # LR scheduling
        model.alpha -= 0.00002

    model.save(curr_model_name)
    print("Model {} Saved".format(curr_model_name))

except AssertionError:
    print("Existing Model {} Found".format(curr_model_name))

Existing Model trial_9.model Found


In [30]:
from gensim.models.doc2vec import Doc2Vec
from nltk.tokenize.treebank import TreebankWordDetokenizer
detokenizer = TreebankWordDetokenizer()

model= Doc2Vec.load(curr_model_name)
print("Model {} Loaded".format(curr_model_name))

Model trial_9.model Loaded


In [35]:
from collections import Counter

def get_doc2vec_encoding(occ, steps=256, alpha=0.03):
    test_data = word_tokenize(occ)
    test_vector = model.infer_vector(test_data, steps=steps, alpha=alpha)
    return test_vector

def get_occ_and_code_from_tokens(training_doc):
    """
    Return the train input in readable form as well as its corresponding NOC code
    """
    tokens = tagged_data[int(training_doc[0])][0]
    
    detokenized_job = detokenizer.detokenize(tokens).replace(" )", ")")
    
#     print('***"{}", "{}"'.format(tokens, detokenized_job))        
#     try:
    code = int(train_df[train_df['input'] == detokenized_job]['code'])
#         print("***", train_df[train_df['input'] == detokenized_job]['code'])
#         try: # TEMPORARY WHILE WE FIX PARENTHESES PROBLEM
#             code = int(
#                 train_df.loc[train_df['input'].str.contains(detokenized_job.split('(')[0]), 'code'].values[0]
#             )
#         except ValueError:
#             code = 0
#         except IndexError:
#             code = 0
    
    return detokenized_job, code

def infer(str_input, verbose=False):
    
    job_vector = get_doc2vec_encoding(str_input)
    
    # to find most similar doc using tags
    similar_doc = model.docvecs.most_similar([job_vector])
    
    codes = []
    
    if verbose:
        print('---------Test on {}---------'.format(str_input))
    
    for doc in similar_doc:
        
        job, code = get_occ_and_code_from_tokens(doc)
                
        codes.append(code)
        
        if verbose:
            print('{} - {}'.format(job, code))
    
    return Counter(codes)

def process_counter(counter):
    
    if len(counter) >= 3:
        v1, v2, v3 = (int(w) for w, c in counter.most_common(3))
        
    elif len(counter) == 2:
        v1, v2  = (int(w) for w, c in counter.most_common(2))
        v3 = 0
        
    elif len(counter) == 1:
        v1 = counter.most_common(1)[0][0]
        v2, v3 = 0, 0
            
    return pd.Series([v1, v2, v3])

def infer_and_vote(occ, verbose=False):
    counter = infer(occ, verbose=verbose)
    return process_counter(counter)

In [36]:
test_occupations = ['doctor', 'athlete', 'member of parliament',
                    'teacher', 'researcher', 'registered nurse', 
                    'CUSTOMER SERVICE', 'MANAGER OF CLEANING BUSINESS',
                   'CAREGIVER', 'Farm Boss']

for occ in test_occupations: 
    occ = preprocess_text(occ, strip_abbrev = STRIP_ABBREVIATIONS)
    print(infer_and_vote(occ, verbose=True))
    

---------Test on doctor---------
industrial physician - 3112
medical officer of health - 3112
appointment clerk - 1414
osteopathic doctor - 3125
locksmithing expert - 7384
homeopathic practitioner - 3232
doctor of naturopathy - 3125
civil aviation medical officer - 3112
respiratory nurse specialist - 3012
registered psychiatric nurse - 3012
0    3112
1    3125
2    3012
dtype: int64
---------Test on athlete---------
golfer - 5251
literary writer - 5121
professional sportsman - 5251
serial writer - 5121
soprano - 5133
musicians agent - 1123
decorative engraver - 5244
custom furniture caner - 5244
automobile racer - 5251
designer and script writer - 5121
0    5251
1    5121
2    5244
dtype: int64
---------Test on member of parliament---------
fishing vessel captain - 8261
fishboat deckhand - 8441
fishing vessel checkerman - 8441
fishing vessel netmender - 8441
fishing vessel skiffwoman - 8441
fishing vessel skiffman - 8441
human resources assistant - 1415
staffing assistant - 1415
sand f

In [39]:
train_df.loc[train_df['input'].str.contains('customer service')]

,input,code
1696,customer service managing supervisor,621
1708,customer service manager automobiles,621
1710,customer service manager retail,621
8433,airline customer service instructor,4021
11826,customer service supervisor retail,6211
12135,customer service clerks supervisor,6314
12139,customer service representative supervisor,6314
12141,customer service representatives supervisor fi...,6314
12678,customer service sales clerk,6421
12933,customer service director water transportation,6522


# Apply embeddings to training data

In [ ]:
train_d2v_embeddings = train_df['input'].apply(get_doc2vec_encoding)

In [ ]:
def vectorize_embeddings(data):
    return np.array([list(x) for x in np.array(data)])

In [ ]:
train_df['doc2vec_embeddings'] = train_d2v_embeddings

In [ ]:
TARGET_CODE_LENGTH = 4

classifier_input = vectorize_embeddings(train_d2v_embeddings)
classifer_output = np.array(train_df['code'].apply(first_n_digits, args = (TARGET_CODE_LENGTH,)))

# Build preliminary classifiers

In [ ]:
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [ ]:
import time

In [ ]:
plt.plot(sorted(np.divide(classifier_input, 9).flatten()))
print(max(classifier_input.flatten()))
print(min(classifier_input.flatten()))

In [ ]:
SVM = SVC(class_weight='balanced', kernel='linear')

start = time.time()
SVM.fit(classifier_input, classifer_output)
print('SVM training duration: {} seconds'.format(time.time()-start))

In [ ]:
RF = RandomForestClassifier(n_estimators=256, max_depth=128, n_jobs=-1, warm_start=True)

start = time.time()
RF.fit(classifier_input, classifer_output)
print('RF training duration: {} seconds'.format(time.time()-start))

In [ ]:
KNN = KNeighborsClassifier(n_neighbors = 1, n_jobs=-1)

start = time.time()
KNN.fit(classifier_input, classifer_output)
print('KNN training duration: {} seconds'.format(time.time()-start))

# Load Test data

In [ ]:
ATP_data = pd.DataFrame(pd.read_excel('./Data/V5_Run Input(1).xlsx'))

In [ ]:
trimmed_ATP = ATP_data
trimmed_ATP['Current Job Title'] = trimmed_ATP['Current Job Title'].apply(preprocess_text, args = (STRIP_ABBREVIATIONS,))
trimmed_ATP['Current Industry'] = trimmed_ATP['Current Industry'].apply(preprocess_text, args = (STRIP_ABBREVIATIONS,))
trimmed_ATP['code'] = trimmed_ATP['NOC code '].apply(lambda x: int(x.strip('\''))).apply(first_n_digits, args=(TARGET_CODE_LENGTH,))
trimmed_ATP.drop(columns = ['NOC code '], inplace = True)

In [ ]:
trimmed_ATP['vote1'], trimmed_ATP['vote2'], trimmed_ATP['vote3'] = None, None, None
trimmed_ATP[['vote1', 'vote2' ,'vote3']] = trimmed_ATP['Current Job Title'].apply(infer_and_vote)
TPs = trimmed_ATP.apply(lambda row: int(row['code']) in [row['vote1'], row['vote2'], row['vote3']], axis = 1)

In [ ]:
train_df.loc[train_df['input'].str.contains('manager')][['input', 'code']]

In [ ]:
df.loc[df['Noc_code'] == "7201"][['group_title', 'Noc_code']]

In [ ]:
trimmed_ATP.sample(10)

In [ ]:
TPs.value_counts()

In [ ]:
test_d2v_embeddings = trimmed_ATP['Current Job Title'].apply(get_doc2vec_encoding)
trimmed_ATP['doc2vec_embeddings'] = test_d2v_embeddings

In [ ]:
vectorized_embeddings = vectorize_embeddings(test_d2v_embeddings)

In [ ]:
trimmed_ATP['rf_pred'] = RF.predict(vectorized_embeddings)

In [ ]:
trimmed_ATP['knn_pred'] = KNN.predict(vectorized_embeddings)

In [ ]:
trimmed_ATP['svm_pred'] = SVM.predict(vectorized_embeddings)

In [ ]:
trimmed_ATP[['knn_pred', 'svm_pred', 'rf_pred', 'code']]

In [ ]:
from sklearn.metrics import accuracy_score, f1_score

for classifier in ['knn','svm', 'rf']:
    print('{} acc:{}, f1-macro:{}'.format(classifier.upper(), 
                                    accuracy_score(
                                        trimmed_ATP['{}_pred'.format(classifier)], 
                                        trimmed_ATP['code']
                                    ),
                                    f1_score(
                                        trimmed_ATP['{}_pred'.format(classifier)],
                                        trimmed_ATP['code'], average = 'macro')
                                   )
     )

In [ ]:
# svm accuracy still tanks, potentially overfitting. the problem is too many output classes. 
# to mitigate, build hierarchical model